## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-15-44-01 +0000,bbc,Trump says EU and Mexico face 30% tariff from ...,https://www.bbc.com/news/articles/cyvj13d9ylpo
1,2025-07-12-15-29-35 +0000,nypost,NYC Councilman and Central Park 5 figure Yusuf...,https://nypost.com/2025/07/12/us-news/yusuf-sa...
2,2025-07-12-15-18-07 +0000,nypost,Prominent Georgia Republican Brant Frost IV ap...,https://nypost.com/2025/07/12/us-news/georgia-...
3,2025-07-12-15-10-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
4,2025-07-12-15-08-29 +0000,nypost,Everything we know about the female Secret Ser...,https://nypost.com/2025/07/12/us-news/heres-wh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,11
204,air,6
6,nyc,5
376,canada,5
249,india,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-07-12-15-10-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,37
54,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,29
10,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...,28
45,2025-07-11-23-05-58 +0000,bbc,Canada's Carney talked tough on Trump - now so...,https://www.bbc.com/news/articles/cvg81518yyyo,23
27,2025-07-12-10-47-08 +0000,bbc,'Was this avoidable?': Families of Air India c...,https://www.bbc.com/news/articles/c5ylv04r1eyo,23


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,37,2025-07-12-15-10-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
27,23,2025-07-12-10-47-08 +0000,bbc,'Was this avoidable?': Families of Air India c...,https://www.bbc.com/news/articles/c5ylv04r1eyo
22,22,2025-07-12-12-52-22 +0000,nypost,Foul! Mets logo ‘illegally’ co-opted to promot...,https://nypost.com/2025/07/12/us-news/foul-met...
14,18,2025-07-12-13-26-21 +0000,nypost,Brooklyn DA rules on criminal charges against ...,https://nypost.com/2025/07/12/us-news/king-of-...
36,18,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
10,17,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...
9,16,2025-07-12-14-24-32 +0000,nypost,Court throws out plea deal for 9/11 mastermind...,https://nypost.com/2025/07/12/us-news/court-th...
11,15,2025-07-12-14-00-00 +0000,nypost,Christians United for Israel offers support wh...,https://nypost.com/2025/07/12/us-news/christia...
37,15,2025-07-12-01-12-57 +0000,bbc,'They were just kids': Mother mourns sons kill...,https://www.bbc.com/news/articles/cy9xgrrq54go
54,14,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
